In [21]:
import numpy as np
#import pydot
from IPython.display import Image
import matplotlib.pyplot as plt
import sklearn.tree as tree

In [22]:
import pydot

In [23]:
data = pd.read_hdf('../budyko_working_er_sm2.hdf','df')

In [24]:
# define a function for computing the EI from a given DI
def budyko(DI):
    return np.sqrt(DI*np.tanh(1/DI)*(1-np.exp(-DI)))

In [25]:
data['PETpenvic_P'] = data['PETpenmanvic']/data['P'] # compute the dryness index
data['ET_P'] = data['ET']/data['P'] # compute the evaporative index
data['pred_EI'] = data['PETpenvic_P'].apply(budyko) # compute the predicted evaportative index for each cell given its aridity
data['Q_P'] = 1.-data['ET_P'] # compute actual runoff for each cell 
data['Qpred_P'] = 1.-data['pred_EI'] # compute predicted runoff for each cell
data['Qanom_P'] = data['Q_P']-data['Qpred_P'] # compute the anomoly (actual - predicted)

In [26]:
#n = 40000 # classifier sample size

In [27]:
#grab = np.random.randint(0,len(data),n)

In [28]:
#samp = data.iloc[grab]

In [29]:
m_nodes = 4

In [30]:
datacolnames = ['Sf','meanpeakswe','infilt','l1_quartz','l3_Wpwp_FRACT','l3_bulkD',
           'l3_init_moist','annual_prec','l2_soil_density','l2_Ksat','rough','l2_Wcr_FRACT','l2_depth',
            'l2_quartz','l1_phi_s','l3_bubble','l1_expt','Ws','l1_Ksat','l1_soil_density','l1_resid_moisture',
            'l2_phi_s','l2_init_moist','l2_expt','l1_init_moist','l1_depth','l3_expt','l3_depth','Dsmax',
            'l1_Wpwp_FRACT','l3_resid_moisture','avg_T','l3_soil_density','l3_phi_s','dp','c','l2_bulkD',
            'l2_bubble','l2_Wpwp_FRACT','l3_Ksat','l3_quartz','Ds','l1_bulkD','l1_bubble','l2_resid_moisture',
           'snowmelt','rain','MAT','prate','rr','DEMelev',]

In [31]:
samp_data =np.zeros([len(data),len(datacolnames)])

In [32]:
col = 0
for key in datacolnames:
    tmp = data[key].as_matrix()
    samp_data[:,col] = tmp
    col += 1

In [33]:
np.shape(samp_data)

(19983, 51)

In [34]:
res = data['Qanom_P'].as_matrix()

In [35]:
res

array([ 0.00499148,  0.00072983,  0.00191586, ..., -0.04165021,
       -0.07618142, -0.10549719])

In [36]:
res = np.reshape(res,[len(res),1])

In [37]:
# fit the tree
clf = tree.DecisionTreeRegressor(max_leaf_nodes=m_nodes,)
clf = clf.fit(data[datacolnames],res)
tree.export_graphviz(clf, out_file='./dot_data',feature_names=datacolnames,filled=True,rounded=True,special_characters=False)

In [38]:
graph = pydot.graph_from_dot_file('./dot_data')
graph.write('./test2.eps', format='eps')

True

In [40]:
ls

Iris example.ipynb*           livneh classifier test.ipynb* test_tree.dot*
Tree tutorial.ipynb*          test.eps*                     tree.png*
dot_data*                     test.pdf*


In [32]:
from sklearn.externals.six import StringIO
dot_data = StringIO()  
tree.export_graphviz(clf, out_file=dot_data,feature_names=datacolnames,filled=True,rounded=True,special_characters=False)  
graph = pydot.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_pdf())  

InvocationException: Program terminated with status: 1. stderr follows: Format: "pdf" not recognized. Use one of: canon cmap cmapx cmapx_np dot eps fig gv imap imap_np ismap pic plain plain-ext pov ps ps2 svg svgz tk vml vmlz xdot xdot1.2 xdot1.4


In [28]:
pred = clf.predict(samp[datacolnames])
resid = res-pred

In [30]:
SST = np.sum((res-np.mean(res))**2)
SSE = np.sum(resid**2)

print 1-(SST/SSE)

0.999985406556


In [32]:
SST

396.47779588106181

In [69]:
SST

198.39462337587207

In [70]:
SSE

7167700.5522641074